<a href="https://colab.research.google.com/github/NK-Mikey/Data_Analysis/blob/main/Project_AR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Automate Report Process**

This project automates the end-to-end creation and delivery of a financial analytics report using Python. The system pulls real-time market data, processes it through a reproducible data pipeline, generates professional visualizations and risk metrics, and automatically emails a PDF report on a scheduled basis.

## 1. GDrive Connection

In [ ]:
# Connect google drive for storage
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

Connecting google drive for storing the pdf report output from this notebook.



## 2. Configuration

In [36]:
# Configure Tickers & Weights
TICKERS = ["AAPL", "MSFT", "SPY"] # Portfolio + Benchmark
WEIGHTS = {"AAPL": 0.4, "MSFT": 0.4, "SPY": 0.2} # Ensuring all the weights sum to 1
LOOKBACK_DAYS = 365 * 3 # Length of data retrival
REPORT_DIR = "/content/drive/MyDrive/Portfolio_reports"

1. We are choosing Apple, Microsoft, and SPY as the portfolio stocks.

    > SPY is an exchange-traded fund (ETF) that tracks the S&P 500 index, which represents 500 of the largest publicly traded companies in the U.S.

2. We assign 40% to Apple, 40% to Microsoft, and 20% to SPY so the script can calculate how much each stock contributes to our portfolio’s total performance.

3. We limit how many days of stock prices we download. In this case, we only retrieve the last 3 years of data.

4. We save the final PDF report from this project into Google Drive.

In [ ]:
# Email Configuration (Colab Testing Only)
from getpass import getpass
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_USER = "your_email@gmail.com"
EMAIL_PASS = getpass("Email App Password: ")
RECEIVER_EMAIL = "receiver_email@gmail.com"

Email App Password: ··········


1. Simple Mail Transfer Protocol (SMTP) is the standard protocol used to send emails either from a client to an email server or from one server to another.

    > Under the hood, Transmission Control Protocol (TCP) and Transport Layer Security (TLS) work together: TCP ensures the email delivered reliably and in the correct order while TLS encrypts the connection so the data remains secure and protected from eavesdropping.

2. A port is a virtual “door” on a server that handles specific types of network traffic. Each port is assigned to a particular protocol or service, ensuring data reaches the correct destination.

In [ ]:
# Testing to see if the email configuration works
import smtplib
from email.message import EmailMessage

# Create the Email
Msg = EmailMessage()
Msg["Subject"] = "Colab SMTP Test"
Msg["From"] = EMAIL_USER
Msg["To"] = RECEIVER_EMAIL
Msg.set_content("This is a test email sent from Google Colab.")

# Send the Email
try:
  server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
  server.starttls()
  server.login(EMAIL_USER, EMAIL_PASS)
  server.send_message(Msg)
  server.quit()
  print("Email sent successfully.")
except Exception as e:
  print("Error sending email:", e)

Email sent successfully.


1. Using the EmailMessage class, we create the email for testing purpose in this environment before integrating into the GitHub Actions for full automation.

    > Note: Always call `starttls` before sending credentials on to the port.

2. Any exception during connect/Auth/send is caught and printed with `except Exception as e` to diagnose the problem.



## 3. Data Ingestion

In [37]:
# Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# Create a directory for reports
os.makedirs(REPORT_DIR, exist_ok=True)

# Define the date range
end = datetime.now()
start = end - timedelta(days = LOOKBACK_DAYS)

# Define a function to fetch stock prices
def fetch_prices(tickers, start, end):
  data = {}
  for t in tickers:
    df = yf.download(t, start = start.strftime("%Y-%m-%d"),
                     end = end.strftime("%Y-%m-%d"),
                     progress = False, auto_adjust = True)
    if df.empty:
      print(f"Warning: no data for {t}")
    else:
        data[t] = df
  return data

# Fetch prices using the function
prices = fetch_prices(TICKERS, start, end)

# Quick sanity check
len(prices), list(prices.keys())

(3, ['AAPL', 'MSFT', 'SPY'])

1. The financial stock price data is retrieved from Yahoo Finance using the Python library `yfinance`.

2. The `os` module is used to create a folder for storing outputs (such as reports and charts), only if the folder does not already exist.

3. A date range is created by calculating the start date and end date using `datetime` and `timedelta`.

   > `datetime` provides the current date and time, while `timedelta` is used to perform date arithmetic, such as calculating the difference between two dates.

4. A function is defined to fetch historical stock price data from Yahoo Finance for the predefined tickers within the specified date range. The function also checks whether any ticker returns empty data. If valid data is retrieved, it is stored in a dictionary named `data`.

   > `auto_adjust=True` ensures that the fetched price data is automatically adjusted for stock splits and dividends, which is suitable for accurately calculating financial KPIs over time.

5. The function is then called, and a quick validation is performed by checking the number of tickers retrieved and listing their keys to ensure they match the predefined tickers.



## 4. Validation & Processing

In [38]:
# Define a function to extract only Close price for all tickers
def validate_and_align(prices_dict):

    # Create a dictionary of Close price series for each ticker
    close_dict = {}
    for ticker, df in prices_dict.items():

        # Extract Close column based on column type
        if isinstance(df.columns, pd.MultiIndex):
            # MultiIndex columns
            if "Close" in df.columns.get_level_values(0):
                close_dict[ticker] = df.xs("Close", level=0, axis=1)[ticker]
            else:
                print(f"{ticker} has no 'Close' in MultiIndex columns")
        else:
            # Single-level columns
            if "Close" in df.columns:
                close_dict[ticker] = df["Close"]
            else:
                print(f"{ticker} has no 'Close' column")

    # Convert dictionary to a single DataFrame
    close_df = pd.DataFrame(close_dict)

    # Drop rows where all values are NaN/empty
    close_df = close_df.dropna(how="all")

    # Filling small gaps using forward and backward fills
    close_df = close_df.ffill().bfill()

    return close_df

# Calling the function to extract only the Close prices for all tickers
close_prices = validate_and_align(prices)

# Display the dataframe
close_prices.tail()

,AAPL,MSFT,SPY
Date,,,
2025-12-22,270.970001,484.920013,684.830017
2025-12-23,272.359985,486.850006,687.960022
2025-12-24,273.809998,488.019989,690.380005
2025-12-26,273.399994,487.709991,690.309998
2025-12-29,273.760010,487.100006,687.849976


1. A function is defined to extract the “Close” price for each ticker from the `prices` dictionary and combine them into a single DataFrame.
    > The function is designed to handle both MultiIndex and single-level columns returned by `yfinance`. When the data uses a MultiIndex, the `.xs()` (cross-section) method is used to extract the `"Close"` price from the appropriate index level.
2. Rows where all ticker values are missing are removed using `dropna(how="all")`.
3. Fill small gaps here and there using forward and backward fills.
    > Any small gaps in the data are handled using forward fill (`ffill`) and backward fill (`bfill`). Forward fill propagates the last available value forward, while backward fill fills missing values at the beginning by using the next available value.
4. Finally, the last five observations are displayed to validate the output.

## 5. Computing Returns, Portfolio Returns and Metrics

In [39]:
# Convert close prices into daily returns
rets = close_prices.pct_change().dropna()

# Create portfolio returns using weights
weights_vector = np.array([WEIGHTS.get(t, 0.0) for t in rets.columns])
port_rets = rets.dot(weights_vector)

# Define trading days
TRADING_DAYS = 252 # U.S. markets trade about 252 days per year

# Create a function to compute Annualized Return
def annualized_return(series):
  cumulative = (1 + series).prod()
  n = len(series) / TRADING_DAYS
  return cumulative ** (1/n) - 1

# Create a function to compute Annualized volatility (risk)
def annualized_vol(series):
  return series.std() * np.sqrt(TRADING_DAYS)

# Create a function to compute Sharpe Ratio (risk-adjusted return)
def sharpe_ratio(series, risk_free = 0.0):
  ar = annualized_return(series)
  avol = annualized_vol(series)
  if avol == 0:
    return np.nan
  return (ar - risk_free) / avol

# Create a function to compute Maximum Drawdown (worst loss)
def max_drawdown(series):
  cum = (1 + series).cumprod()
  running_max = cum.cummax()
  drawdown = (cum - running_max)/running_max
  return drawdown.min()

# Create a function to compute Value at Risk (VaR)
def var_historic(series, level = 0.95):
  return -np.percentile(series.dropna(), (1 - level)* 100)

# Create a function to compute Sortino Ratio (downside risk only)
def sortino_ratio(series, risk_free = 0.0):
  neg_rets = series[series < 0]
  downside_std = neg_rets.std() * np.sqrt(TRADING_DAYS)
  ar = annualized_return(series)
  if downside_std == 0:
    return np.nan
  return (ar - risk_free) / downside_std

# Computing summarized portfolio metrics
metrics = {
    "portfolio_annual_return": annualized_return(port_rets),
    "portfolio_annual_vol": annualized_vol(port_rets),
    "portfolio_sharpe": sharpe_ratio(port_rets),
    "portfolio_sortino": sortino_ratio(port_rets),
    "portfolio_max_drawdown": max_drawdown(port_rets),
    "portfolio_VaR_95": var_historic(port_rets, level = 0.95),
    "portfolio_VaR_99": var_historic(port_rets, level = 0.99),
}

display_names_metrics = {
    "portfolio_annual_return": "Annual Return",
    "portfolio_annual_vol": "Annual Volatility",
    "portfolio_sharpe": "Sharpe Ratio",
    "portfolio_sortino": "Sortino Ratio",
    "portfolio_max_drawdown": "Maximum Drawdown",
    "portfolio_VaR_95": "Value at Risk (95%)",
    "portfolio_VaR_99": "Value at Risk (99%)",
}

# Computing per-asset metrics for comparison between tickers
asset_metrics = {}
for t in rets.columns:
  s = rets[t]
  asset_metrics[t] = {
      "annual_return": annualized_return(s),
      "annual_vol": annualized_vol(s),
      "sharpe": sharpe_ratio(s),
      "max_drawdown": max_drawdown(s),
  }

display_names_asset = {
    "annual_return": "Annual Return",
    "annual_vol": "Annual Volatility",
    "sharpe": "Sharpe Ratio",
    "max_drawdown": "Maximum Drawdown",
}

# converting the dictionaries to dataframe for better readability
metrics_df = pd.DataFrame.from_dict(metrics, orient="index", columns=["Value"]).rename(index=display_names_metrics)
asset_df = pd.DataFrame(asset_metrics).rename(index=display_names_asset).T

# Displaying the data frames
from IPython.display import display, Markdown

display(Markdown("### Portfolio-Level Metrics"))
display(metrics_df)

display(Markdown("### Asset-Level Metrics"))
display(asset_df)

### Portfolio-Level Metrics

,Value
Annual Return,0.291747
Annual Volatility,0.193980
Sharpe Ratio,1.504002
Sortino Ratio,2.242303
Maximum Drawdown,-0.249092
Value at Risk (95%),0.018122
Value at Risk (99%),0.029981


### Asset-Level Metrics

,Annual Return,Annual Volatility,Sharpe Ratio,Maximum Drawdown
AAPL,0.308137,0.256324,1.202139,-0.333605
MSFT,0.279779,0.232299,1.204390,-0.237266
SPY,0.236413,0.153632,1.538830,-0.187552
